# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import unicodedata

# ==========================================
# 1. パス設定
# ==========================================
# ノートブックの場所を基準に、一つ上の階層をプロジェクトルートとする
# 例: notebooks/01_create_master.ipynb -> project_root/
BASE_DIR = Path("..").resolve()

# フォルダパス
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

# 出力フォルダ作成
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# 出力ファイル
OUTPUT_FILE = PROCESSED_DIR / "master_nurse_turnover_v2.csv"

print(f"📂 Project Root: {BASE_DIR}")
print(f"📂 Raw Data Dir: {RAW_DIR}")

📂 Project Root: /Users/hideomi.h/nurse-turnover-analysis
📂 Raw Data Dir: /Users/hideomi.h/nurse-turnover-analysis/data/raw


In [15]:
# ==========================================
# 2. ファイル存在確認
# ==========================================
# 必要なファイル一覧
FILES = {
    "turnover": RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv",
    "night_shift": RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv",
    "nurse_count": RAW_DIR / "厚労省_衛生行政報告例_看護師数_2023.csv",
    "hospital": RAW_DIR / "厚労省_医療施設調査_病床規模_2024.csv",
    "rent": RAW_DIR / "総務省_住宅土地統計調査_家賃_2024.xlsx",
    "density": RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"
}

missing = []
for key, path in FILES.items():
    if not path.exists():
        missing.append(path.name)
        print(f"❌ Missing: {path.name}")
    else:
        print(f"✅ Found: {path.name}")

if missing:
    raise FileNotFoundError(f"以下のファイルが見つかりません: {missing}\nフォルダ構成を確認してください。")

✅ Found: 日本看護協会_離職率_都道府県別_2023.csv
✅ Found: 日本看護協会_夜勤72h超過率_都道府県別_2024.csv
✅ Found: 厚労省_衛生行政報告例_看護師数_2023.csv
✅ Found: 厚労省_医療施設調査_病床規模_2024.csv
✅ Found: 総務省_住宅土地統計調査_家賃_2024.xlsx
✅ Found: 総務省_社会生活統計指標_人口密度_2023.csv


In [16]:
# ==========================================
# 4. 離職率・年収データ (Base Master)
# ==========================================
print("🔄 Loading Turnover & Income Data...")

df_turnover = pd.read_csv(FILES["turnover"])

# ★修正ポイント: 列名の強制リネーム（日本語 -> 英語）
# 元ファイルの列名が多少違っても動くように、想定される名前をマッピングします
rename_map = {
    "都道府県": "prefecture",
    "離職率": "turnover_total",
    "正規雇用職員離職率": "turnover_total",
    "新卒離職率": "turnover_new_grad",
    "既卒離職率": "turnover_experienced",
    "税込年収": "annual_income",
    "年収": "annual_income",
    "給与": "annual_income"
}
df_turnover = df_turnover.rename(columns=rename_map)

# 正規化
df_turnover["prefecture"] = df_turnover["prefecture"].apply(normalize_pref)

# 【重要】年収の単位変換（千円 -> 万円）
if "annual_income" in df_turnover.columns:
    # データが文字列になっていないか確認してから割り算
    df_turnover["annual_income"] = pd.to_numeric(df_turnover["annual_income"], errors="coerce")
    # 明らかに千円単位（1000以上）の場合のみ割る
    if df_turnover["annual_income"].mean() > 1000:
        df_turnover["annual_income"] = df_turnover["annual_income"] / 10
        print("   -> Converted Annual Income unit (Thousands -> Man-yen)")
else:
    print("⚠️ Warning: 'annual_income' column not found. Check CSV headers.")
    print(f"   Columns: {list(df_turnover.columns)}")

# マスタとなる都道府県リストを作成
MASTER_PREFS = set(df_turnover["prefecture"].unique())

print(f"   -> Loaded {len(df_turnover)} rows.")

🔄 Loading Turnover & Income Data...
⚠️ Warning: 'annual_income' column not found. Check CSV headers.
   Columns: ['prefecture', 'turnover_total', 'turnover_new_grad', 'turnover_experienced']
   -> Loaded 48 rows.


In [17]:
# ==========================================
# 4. 離職率・年収データ (Base Master)
# ==========================================
print("🔄 Loading Turnover & Income Data...")

df_turnover = pd.read_csv(FILES["turnover"])
df_turnover["prefecture"] = df_turnover["prefecture"].apply(normalize_pref)

# 【重要】年収の単位変換（千円 -> 万円）
# 元データが「4789(千円)」の場合 -> 「478.9(万円)」にする
if "annual_income" in df_turnover.columns:
    df_turnover["annual_income"] = df_turnover["annual_income"] / 10

# マスタとなる都道府県リストを作成
MASTER_PREFS = set(df_turnover["prefecture"].unique())

print(f"   -> Loaded {len(df_turnover)} rows.")

🔄 Loading Turnover & Income Data...
   -> Loaded 48 rows.


In [18]:
# ==========================================
# 5. 夜勤負担データ
# ==========================================
print("🔄 Loading Night Shift Data...")

df_night = pd.read_csv(FILES["night_shift"])
df_night["prefecture"] = df_night["prefecture"].apply(normalize_pref)

print(f"   -> Loaded {len(df_night)} rows.")

🔄 Loading Night Shift Data...
   -> Loaded 48 rows.


In [19]:
# ==========================================
# 6. 看護師数データ (厚労省)
# ==========================================
print("🔄 Loading Nurse Count Data...")

# header=3 (4行目) から読み込む
df_nurse = pd.read_csv(FILES["nurse_count"], header=3, encoding="cp932")

# 1列目(都道府県)と4列目(総数)を抽出
df_nurse = df_nurse.iloc[:, [0, 3]]
df_nurse.columns = ["prefecture", "nurse_count"]

# 正規化とクリーニング
df_nurse["prefecture"] = df_nurse["prefecture"].apply(normalize_pref)
df_nurse["nurse_count"] = df_nurse["nurse_count"].apply(clean_number)

# マスタにある都道府県のみ残す
df_nurse = df_nurse[df_nurse["prefecture"].isin(MASTER_PREFS)]

print(f"   -> Loaded {len(df_nurse)} rows.")

🔄 Loading Nurse Count Data...
   -> Loaded 47 rows.


In [20]:
# ==========================================
# 7. 病院数・病床規模 (★最重要ロジック)
# ==========================================
print("🔄 Loading Hospital Data (Robust Logic)...")

# 列ズレ防止のため、強制的に100列確保して読み込む
df_hosp_raw = pd.read_csv(FILES["hospital"], encoding="utf-8-sig", header=None, dtype=str, names=range(100))

# 必要な列を抽出
# Col 8 (I列): 都道府県名
# Col 9 (J列): コード (1=総数, 11-15=大規模)
# Col 13 (N列): 施設数
df_wk = pd.DataFrame({
    "pref_raw": df_hosp_raw[8],
    "code": df_hosp_raw[9],
    "val_raw": df_hosp_raw[13]
})

# 1. 都道府県名の特定（マスタとの照合）
def resolve_pref(text):
    text = normalize_pref(text)
    if not text: return None
    # "全国" "再掲" などを除外
    if text in ["全国", "総数", "再掲"]: return None
    # 前方一致で都道府県を特定
    for p in MASTER_PREFS:
        if p.startswith(text): return p
    return None

df_wk["pref_key"] = df_wk["pref_raw"].apply(resolve_pref)

# 2. ffillで内訳行に都道府県名をコピー
df_wk["prefecture"] = df_wk["pref_key"].ffill()

# 3. 有効な行（都道府県がある行）のみ抽出
df_wk = df_wk.dropna(subset=["prefecture"])

# 4. 数値化
df_wk["count"] = df_wk["val_raw"].apply(clean_number)
df_wk["code"] = pd.to_numeric(df_wk["code"], errors="coerce")

# 5. 集計
# 【病院総数】 Code == 1
df_total = df_wk[df_wk["code"] == 1].groupby("prefecture")["count"].sum().reset_index()
df_total.columns = ["prefecture", "hospital_count"]

# 【大規模病院数】 Code in [11, 12, 13, 14, 15] (500床以上)
large_codes = [11, 12, 13, 14, 15]
df_large = df_wk[df_wk["code"].isin(large_codes)].groupby("prefecture")["count"].sum().reset_index()
df_large.columns = ["prefecture", "large_hospital_count"]

# 結合
df_hospital = pd.merge(df_total, df_large, on="prefecture", how="left").fillna(0)

# 検証（北海道と沖縄）
print("   -> Validation:")
for p in ["北海道", "沖縄県"]:
    if p in df_hospital["prefecture"].values:
        row = df_hospital[df_hospital["prefecture"]==p].iloc[0]
        print(f"      {p}: Total={row['hospital_count']}, Large={row['large_hospital_count']}")

🔄 Loading Hospital Data (Robust Logic)...
   -> Validation:
      北海道: Total=534, Large=17
      沖縄県: Total=3753, Large=240


In [21]:
# ==========================================
# 7. 病院数・病床規模 (★修正版ロジック)
# ==========================================
print("🔄 Loading Hospital Data (Fixed Logic)...")

# 強制的に100列確保
df_hosp_raw = pd.read_csv(FILES["hospital"], encoding="utf-8-sig", header=None, dtype=str, names=range(100))

df_wk = pd.DataFrame({
    "pref_raw": df_hosp_raw[8],  # I列
    "code": df_hosp_raw[9],      # J列
    "val_raw": df_hosp_raw[13]   # N列
})

# 1. 都道府県名の特定（ストッパー機能付き）
def resolve_pref(text):
    text = normalize_pref(text)
    if not text: return None
    # ★ここが修正点: 「全国」や「計」が来たら 'STOP' というマーカーを返す
    if any(x in text for x in ["全国", "総数", "再掲", "計"]):
        return "STOP"
    # 都道府県とのマッチング
    for p in MASTER_PREFS:
        if p.startswith(text): return p
    return None

df_wk["pref_key"] = df_wk["pref_raw"].apply(resolve_pref)

# 2. ffill（STOPマーカーも含めて埋める）
df_wk["prefecture"] = df_wk["pref_key"].ffill()

# 3. STOPマーカーやゴミ行を除外
# "STOP" になった行（＝全国計など）とその配下の行をここで捨てる
df_wk = df_wk[df_wk["prefecture"].isin(MASTER_PREFS)]

# 4. 数値化 & 集計
df_wk["count"] = df_wk["val_raw"].apply(clean_number)
df_wk["code"] = pd.to_numeric(df_wk["code"], errors="coerce")

# 総数 (Code=1)
df_total = df_wk[df_wk["code"] == 1].groupby("prefecture")["count"].sum().reset_index()
df_total.columns = ["prefecture", "hospital_count"]

# 大規模 (Code=11~15)
large_codes = [11, 12, 13, 14, 15]
df_large = df_wk[df_wk["code"].isin(large_codes)].groupby("prefecture")["count"].sum().reset_index()
df_large.columns = ["prefecture", "large_hospital_count"]

# 結合
df_hospital = pd.merge(df_total, df_large, on="prefecture", how="left").fillna(0)

# 検証（沖縄が 89 になれば成功）
print("   -> Validation:")
for p in ["北海道", "沖縄県"]:
    if p in df_hospital["prefecture"].values:
        row = df_hospital[df_hospital["prefecture"]==p].iloc[0]
        print(f"      {p}: Total={row['hospital_count']}, Large={row['large_hospital_count']}")

🔄 Loading Hospital Data (Fixed Logic)...
   -> Validation:
      北海道: Total=534, Large=17
      沖縄県: Total=89, Large=2


In [22]:
# ==========================================
# 9. 人口・人口密度
# ==========================================
print("🔄 Loading Density Data...")

# 13行目から読み込み
df_dens_raw = pd.read_csv(FILES["density"], encoding="utf-8-sig", header=12)

# カラム名で抽出（変動に対応）
col_pref = [c for c in df_dens_raw.columns if "地域" in c][0]
col_pop = [c for c in df_dens_raw.columns if "総人口" in c][0] # 万人単位
col_dens = [c for c in df_dens_raw.columns if "密度" in c][0]

df_density = df_dens_raw[[col_pref, col_pop, col_dens]].copy()
df_density.columns = ["prefecture", "population", "population_density"]

df_density["prefecture"] = df_density["prefecture"].apply(normalize_pref)
for c in ["population", "population_density"]:
    df_density[c] = df_density[c].apply(clean_number)

print(f"   -> Loaded {len(df_density)} rows.")

🔄 Loading Density Data...
   -> Loaded 48 rows.


In [23]:
# ==========================================
# 10. データ結合と指標計算
# ==========================================
print("🔄 Merging all datasets...")

# 離職率データをベースにする
df_master = df_turnover.copy()

# 順次マージ
merge_list = [df_night, df_nurse, df_hospital, df_rent, df_density]
for df in merge_list:
    df_master = pd.merge(df_master, df, on="prefecture", how="left")

# --- 計算指標 ---

# 1. 大規模病院比率 (%)
# 0除算回避
df_master["large_hospital_ratio"] = df_master.apply(
    lambda x: (x["large_hospital_count"] / x["hospital_count"] * 100) if x["hospital_count"] > 0 else 0,
    axis=1
)

# 2. 人口あたり係数 (Population Unit)
# 人口は「万人」単位なので、10で割ると「10万人単位」になる
# 例: 100万人 / 10 = 10 (10万が10個)
df_master["pop_unit_100k"] = df_master["population"] / 10

# 3. 人口10万人あたり看護師数
df_master["nurse_per_100k"] = df_master["nurse_count"] / df_master["pop_unit_100k"]

# 4. 人口10万人あたり病院数
df_master["hospital_per_100k"] = df_master["hospital_count"] / df_master["pop_unit_100k"]

# 不要な一時カラム削除
df_master.drop(columns=["pop_unit_100k"], inplace=True)

print("✅ Merge Complete.")

🔄 Merging all datasets...
✅ Merge Complete.


In [26]:
# ==========================================
# 11. 最終確認と保存
# ==========================================

# 欠損値チェック
null_counts = df_master.isnull().sum()
if null_counts.sum() > 0:
    print("⚠️ Warning: Missing values detected:")
    print(null_counts[null_counts > 0])
else:
    print("✅ No missing values.")

# 沖縄チェック (重要)
if "沖縄県" in df_master["prefecture"].values:
    row = df_master[df_master["prefecture"]=="沖縄県"].iloc[0]
    print("\n🔍 Okinawa Verification:")
    print(f"  Hospital Count: {row['hospital_count']} (Expected: 89)")
    print(f"  Large Hospital: {row['large_hospital_count']} (Expected: 2)")
    print(f"  Income: {row['annual_income']} (Expected: ~471)")

# 保存
# カラム並び替え
cols_order = [
    "prefecture",
    "turnover_total", "turnover_new_grad", "turnover_experienced",
    "annual_income",
    "nurse_count", "nurse_per_100k",
    "hospital_count", "large_hospital_count", "large_hospital_ratio", "hospital_per_100k",
    "rent_private",
    "night_shift_72h_plus",
    "population", "population_density"
]
# 存在するものだけ
cols_final = [c for c in cols_order if c in df_master.columns] + [c for c in df_master.columns if c not in cols_order]

df_master = df_master[cols_final]
df_master.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

print(f"\n✅ SUCCESS! Dataset saved to:\n{OUTPUT_FILE}")

⚠️ Warning: Missing values detected:
nurse_count              1
hospital_count           1
large_hospital_count     1
rent_private             1
population              48
population_density      48
nurse_per_100k          48
hospital_per_100k       48
dtype: int64

🔍 Okinawa Verification:
  Hospital Count: 89.0 (Expected: 89)
  Large Hospital: 2.0 (Expected: 2)


KeyError: 'annual_income'

In [25]:
import pandas as pd
from pathlib import Path

# パス設定（Notebookの設定に合わせます）
BASE_DIR = Path("..").resolve()
RAW_DIR = BASE_DIR / "data" / "raw"

print("=== 診断開始 ===")

# 1. 離職率ファイルの「本当の列名」を確認
# (ここがズレていると 'annual_income' が作られずエラーになります)
print("\n1. 離職率データ (Turnover) の中身確認:")
try:
    target_file = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"
    # 文字コードも疑って標準とShift-JIS両方試す
    try:
        df_t = pd.read_csv(target_file)
        print("   [UTF-8読み込み] 列名一覧:", list(df_t.columns))
    except:
        df_t = pd.read_csv(target_file, encoding="cp932")
        print("   [Shift-JIS読み込み] 列名一覧:", list(df_t.columns))
    print("   先頭1行:\n", df_t.head(1))
except Exception as e:
    print("   ❌ 読み込み失敗:", e)

# 2. 人口ファイルの「読み込まれ方」を確認
# (ここがズレていると人口データが全部NaNになります)
print("\n2. 人口データ (Density) の中身確認:")
try:
    target_file = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"
    # header=12 であっているか確認
    df_d = pd.read_csv(target_file, encoding="utf-8-sig", header=12)
    print("   [Header=12] 列名一覧:", list(df_d.columns))
    print("   先頭1行:\n", df_d.head(1))
except Exception as e:
    print("   ❌ 読み込み失敗:", e)

print("\n=== 診断終了 ===")

=== 診断開始 ===

1. 離職率データ (Turnover) の中身確認:
   [UTF-8読み込み] 列名一覧: ['prefecture', 'turnover_total', 'turnover_new_grad', 'turnover_experienced']
   先頭1行:
   prefecture  turnover_total  turnover_new_grad  turnover_experienced
0        北海道            11.5                5.9                  16.6

2. 人口データ (Density) の中身確認:
   [Header=12] 列名一覧: ['調査年 コード', '調査年 補助コード', '調査年', '地域 コード', '地域 補助コード', '地域', '/Ａ\u3000人口・世帯', '#A011000_総人口【万人】', '#A0110001_総人口（男）【万人】', '#A0110002_総人口（女）【万人】', '#A01101_全国総人口に占める人口割合（A1101/A1101(全国)）【％】', '#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】', '#A0191002_将来推計人口（2025年）【人】', '#A0191003_将来推計人口（2030年）【人】', '#A0191004_将来推計人口（2035年）【人】', '#A0191005_将来推計人口（2040年）【人】', '#A0191006_将来推計人口（2045年）【人】', '#A0191007_将来推計人口（2050年）【人】', '#A02101_人口性比（総数）（A110101/A110102）【‐】', '#A02102_人口性比（15歳未満人口）(A130101/A130102)【‐】', '#A02103_人口性比（15～64歳人口）(A130201/A130202)【‐】', '#A02104_人口性比（65歳以上人口) (A130301/A130302)【‐】', '#A03501_15歳未満人口割合【％】', '#A03502_15～64歳人口割合【％】', '#A03